# HTTP Client Example

This notebook demonstrates how to use the `HTTPRegistryClient` to connect to a remote LangGate service for model information.

In [1]:
import os
from pprint import pprint as pp
from pydantic import SecretStr

# Configure server URL
LANGGATE_URL = "http://localhost:4000/api/v1"

# If running the langgate server behind Envoy proxy in docker:
# LANGGATE_URL = "http://localhost:10000/api/v1"

# If running and accessing langgate from within a kubernetes cluster:
# LANGGATE_URL = "http://langgate.ns.svc.cluster.local:10000/api/v1"

# Optional: Set API key (if your LangGate service requires authentication)
api_key = SecretStr("your_api_key")

## Initialize the HTTP Client

In [2]:
from langgate.client.http import HTTPRegistryClient

# Initialize the HTTP client
# HTTPRegistryClient is a singleton
client = HTTPRegistryClient(base_url=LANGGATE_URL, api_key=api_key)

2025-07-09 23:42:37 [debug    ] creating_http_registry_client_singleton
2025-07-09 23:42:37 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 image_info_cls=<class 'langgate.core.models.ImageModelInfo'> llm_info_cls=<class 'langgate.core.models.LLMInfo'>
2025-07-09 23:42:37 [debug    ] initialized_http_registry_client_singleton


## List Available Models

In [3]:
try:
    # List available LLMs
    llms = await client.list_llms()
    print(f"Available LLMs: {len(llms)}")
    for model in llms[:5]:  # Show first 5 models
        print(f"- {model.id}: {model.name}")

    print("\n" + "=" * 50 + "\n")

    # List available image models
    image_models = await client.list_image_models()
    print(f"Available Image Models: {len(image_models)}")
    for model in image_models[:5]:  # Show first 5 models
        print(f"- {model.id}: {model.name}")
except Exception as e:
    print(f"Error connecting to the LangGate service: {str(e)}")
    print("Note: This example requires a running LangGate service.")

2025-07-09 23:42:41 [debug    ] refreshing_model_caches       
2025-07-09 23:42:41 [debug    ] refreshed_model_caches         image_count=33 llm_count=61
Available LLMs: 61
- openai/gpt-4.1: GPT-4.1
- openai/gpt-4.1-mini: GPT-4.1 mini
- openai/gpt-4.1-nano: GPT-4.1 nano
- openai/gpt-4o: GPT-4o
- openai/gpt-4o-mini: GPT-4o mini


Available Image Models: 33
- openai/gpt-image-1: GPT Image 1
- openai/dall-e-3: DALL-E 3
- xai/grok-2-image: Grok 2 Image
- google/imagen-4-ultra: Imagen 4 Ultra
- google/imagen-4: Imagen 4


## Get Detailed Model Information

In [4]:
try:
    # Get detailed information about a specific language model
    if llms and len(llms) > 0:
        model_id = llms[0].id
        model_info = await client.get_llm_info(model_id)

        print(f"\nModel: {model_info.name}")
        print(f"Provider: {model_info.provider.name}")

        print("\nModel metadata:")
        pp(model_info.model_dump(exclude_none=True))
except Exception as exc:
    print(f"Error fetching model information: {str(exc)}")
    print("Note: This example requires a running LangGate service.")


Model: GPT-4.1
Provider: OpenAI

Model metadata:
{'capabilities': {'supports_parallel_tool_calls': True,
                  'supports_prompt_caching': True,
                  'supports_response_schema': True,
                  'supports_system_messages': True,
                  'supports_tool_choice': True,
                  'supports_tools': True,
                  'supports_vision': True},
 'context_window': {'max_input_tokens': 1047576, 'max_output_tokens': 32768},
 'costs': {'cache_read_input_token_cost': Decimal('5E-7'),
           'input_cost_per_token': Decimal('0.000002'),
           'output_cost_per_token': Decimal('0.000008')},
 'description': "GPT-4.1 is the latest iteration of OpenAI's flagship model "
                'with improved capabilities across all domains.',
 'id': 'openai/gpt-4.1',
 'name': 'GPT-4.1',
 'provider': {'id': 'openai', 'name': 'OpenAI'},
 'provider_id': 'openai',
 'updated_dt': datetime.datetime(2025, 7, 9, 22, 42, 31, 394249, tzinfo=TzInfo(UTC))}


In [5]:
try:
    # Get detailed information about a specific image model
    if image_models and len(image_models) > 0:
        model_id = image_models[0].id
        model_info = await client.get_image_model_info(model_id)

        print(f"\nModel: {model_info.name}")
        print(f"Provider: {model_info.provider.name}")

        print("\nModel metadata:")
        pp(model_info.model_dump(exclude_none=True))
except Exception as exc:
    print(f"Error fetching model information: {str(exc)}")
    print("Note: This example requires a running LangGate service.")


Model: GPT Image 1
Provider: OpenAI

Model metadata:
{'costs': {'image_generation': {'quality_tiers': {'high': {'1024x1024': Decimal('0.167'),
                                                           '1024x1536': Decimal('0.25'),
                                                           '1536x1024': Decimal('0.25')},
                                                  'low': {'1024x1024': Decimal('0.011'),
                                                          '1024x1536': Decimal('0.016'),
                                                          '1536x1024': Decimal('0.016')},
                                                  'medium': {'1024x1024': Decimal('0.042'),
                                                             '1024x1536': Decimal('0.063'),
                                                             '1536x1024': Decimal('0.063')}}},
           'token_costs': {'input_cached_cost_per_token': Decimal('0.0000025'),
                           'input_cost_per_token':